In [12]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.1.0


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
  32/1000 [..............................] - ETA: 29s - loss: 2.3419 - sparse_categorical_accuracy: 0.0625
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1383 - sparse_categorical_accuracy: 0.6800 - val_loss: 0.7244 - val_sparse_categorical_accuracy: 0.7800
Epoch 2/10
  32/1000 [..............................] - ETA: 0s - loss: 0.4485 - sparse_categorical_accuracy: 0.8438
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 124us/sample - loss: 0.4155 - sparse_categorical_accuracy: 0.8920 - val_loss: 0.5216 - val_sparse_categorical_accuracy: 0.8420
Epoch 3/10
  32/1000 [..............................] - ETA: 0s - loss: 0.2093 - sparse_categorical_accuracy: 0.9375
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 135us/sample - loss: 0.2934 - sparse_categorical_accuracy: 0

In [5]:
ls {checkpoint_dir}

 Volume in Laufwerk C: hat keine Bezeichnung.
 Volumeseriennummer: 0654-BAD9

 Verzeichnis von C:\Users\8holz\Dokumente\GitHub\K14\training_1

06.12.2020  10:18    <DIR>          .
06.12.2020  10:18    <DIR>          ..
06.12.2020  10:18                71 checkpoint
06.12.2020  10:18             2'064 cp.ckpt.data-00000-of-00002
06.12.2020  10:18         4'884'624 cp.ckpt.data-00001-of-00002
06.12.2020  10:18             1'251 cp.ckpt.index
05.12.2020  20:42    <DIR>          cp.ckpt_temp_060bdefa8f6f4bb5855e99f1340e8f8b
05.12.2020  20:46    <DIR>          cp.ckpt_temp_4fbf557ff3e74ecbaca98afb11a0a2bf
05.12.2020  20:45    <DIR>          cp.ckpt_temp_736d8ae343fc49ce9d7e2761829df858
               4 Datei(en),      4'888'010 Bytes
               5 Verzeichnis(se), 96'360'820'736 Bytes frei


In [6]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

1000/1000 - 0s - loss: 2.3360 - sparse_categorical_accuracy: 0.1040
Untrained model, accuracy: 10.40%


In [7]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

1000/1000 - 0s - loss: 0.3973 - sparse_categorical_accuracy: 0.8700
Restored model, accuracy: 87.00%


In [15]:
a = open("kk.png", "w+")

In [16]:
os.remove("kk.png")

PermissionError: [WinError 32] Der Prozess kann nicht auf die Datei zugreifen, da sie von einem anderen Prozess verwendet wird: 'kk.png'